# Installing neccessary packages


# Install your packages here

In [118]:
install.packages('jsonlite')
install.packages('rvest')
install.packages('tidyverse')
install.packages('lubridate')
install.packages('httr')
install.packages("fastDummies")
install.packages('tidymodels')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("jsonlite"):
“installation of package ‘jsonlite’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("httr"):
“installation of package ‘httr’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# Load the libraries here

In [141]:
library(jsonlite)
library(httr)
library(rvest)
library(tidyverse)
library(lubridate)
library(fastDummies)
library(tidymodels)

# trimStringArray function

In [2]:
trimStringArray = function(array) {
  if (is.array(array)) {
    for (i in 1:length(array())) {
      array[i] = trimws(array[i])
    }
  } else {
    array = trimws(array)
  }
  array
}

# ternaryOperatorFunction | Vectorized function

In [142]:
ternary = function(expression_1, expression_2, condition) {
    if(condition) {
        expression_1
    } else {
        expression_2
    }
}

ternary_v = Vectorize(ternary)

# Create a vectorized version of strsplit()

In [4]:
strsplit_custom = function (string, splitter, index) {
    strsplit(string, splitter)[[1]][index]
}

strsplit_custom_v = Vectorize(strsplit_custom)

# Scrape Teilnehmer von WM

In [5]:
scrapeColumnsWM = function(game_name) {

  # base url

  url = 'https://www.transfermarkt.com/weltmeisterschaft-2022/teilnehmer/pokalwettbewerb/WM22/saison_id/2021'

  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header
  slugs = c()
  club_ids = c()

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    item %>%
      html_elements('.table-header') %>%
      html_text() %>%
      trimStringArray() -> name

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # scrape the url with the slugs and club ids of the countries

      responsive_table %>%
          html_elements('tr td:first-of-type a') %>%
          html_attr('href') %>%
          strsplit('/') -> raw_url_data
      
      raw_url_data

      # obtain the slug of all countries

      raw_url_data[1][[1]][2]
      raw_url_data[1][[1]][length(raw_url_data[1][[1]])]

      for (i in 1:length(raw_url_data)) {
          slugs = c(slugs, raw_url_data[i][[1]][2])
          club_ids = c(club_ids, raw_url_data[i][[1]][length(raw_url_data[1][[1]])])
      }

      # print slugs

      slugs

      # print club_ids

      club_ids

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames

      # add slug and club_id to columnNames

      columnNames = c(columnNames, c('slug', 'club_id'))

      # print columnNames again

      columnNames

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              print(paste0("Column ", i, " is not a valid value"))
          }

      }

      # add club_id and slug to values

      columnValues = append(columnValues, list(slugs))
      columnValues = append(columnValues, list(club_ids))

      # print columnValues

      columnValues

      # return list
      list(columnNames, columnValues)
  } else {
      # return NULL
      NULL
  }
}

# create Dataframe from scrapeColumnsWM

In [6]:
create_dataframeWM = function(name) {
  df_wm = scrapeColumnsWM(name)

  if(!(is.null(df_wm))) {
      columns = df_wm[[2]]
      columnNames = df_wm[[1]]

      #print(columnNames)

      df_wm = data.frame(columns[[1]])

      for (i in 2:length(columns)) {
          df_wm[i] = columns[[i]]
      }

      names(df_wm) = columnNames[2:length(columnNames)]
      df_wm
  } else {
      NULL
  }
}

In [7]:
df_wm = create_dataframeWM('teilnehmende teams an der wm')
df_wm

[1] "Column 1 is not a valid value"


Club,Squad,&oslash-Age,WC particip.,Foreigners,Market Value,&oslash-Market Value,slug,club_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
England,28,25.6,16,10.7 %,€1.34bn,€47.71m,england,3299
Brazil,26,27.2,22,88.5 %,€1.05bn,€40.54m,brasilien,3439
Portugal,25,27.3,8,84.0 %,€938.00m,€37.52m,portugal,3300
France,23,26.9,16,73.9 %,€909.50m,€39.54m,frankreich,3377
Germany,24,26.6,20,33.3 %,€818.00m,€34.08m,deutschland,3262
Spain,25,25.9,16,28.0 %,€790.00m,€31.60m,spanien,3375
Argentina,28,27.7,18,100.0 %,€671.00m,€23.96m,argentinien,3437
Belgium,30,27.4,14,76.7 %,€564.50m,€18.82m,belgien,3382
Netherlands,24,26.7,11,50.0 %,€536.95m,€22.37m,niederlande,3379


In [8]:
teilnehmer = df_wm$Club
teilnehmer

[1] "England"       "Brazil"        "Portugal"      "France"       
 [5] "Germany"       "Spain"         "Argentina"     "Belgium"      
 [9] "Netherlands"   "Uruguay"       "Croatia"       "Serbia"       
[13] "Denmark"       "Senegal"       "Poland"        "United States"
[17] "Morocco"       "Switzerland"   "Ghana"         "Mexico"       
[21] "Canada"        "Cameroon"      "Wales"         "South Korea"  
[25] "Japan"         "Ecuador"       "Iran"          "Tunisia"      
[29] "Australia"     "Saudi Arabia"  "Qatar"         "Costa Rica"

# Scrape Columns
We wrote a function, that takes a url to scrape all tables

In [9]:
scrapeGame = function(year, country, club_id, game_name) {

  # base url

  url = paste0("https://www.transfermarkt.com/",
               country,
               "/spielplan/verein/",
               club_id,
               "/plus/0?saison_id=", year)
    
  # print url
  print(url)
    
  # get status code
  res = httr::GET(url)

  # read response
  code = httr::status_code(res)
    
  # print status code
  print(code)
    
  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    if(length(html_elements(item, '.table-header') %>% html_children())!= 0) {
      item %>%
        html_elements('.table-header a img') %>%
        html_attr('title') -> name

      #print(name)
    } else {
      item %>%
        html_elements('.table-header') %>%
        html_text() %>%
        trimStringArray() -> name
        
        #print(name)
    }

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames
      
      # add game_name to columnNames
      
      columnNames = append(columnNames, 'game_name')
      columnNames = append(columnNames, 'country')

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              #print(paste0("Column ", i, " is not a valid value"))
          }

      }
        
      # add game_name to columnValues
      
      columnValues = append(columnValues, name)
      
      # add country name to columnValues
      columnValues = append(columnValues, country)
      
      # return data frame
      # change column names by replacing whitespace with underscore
      columnNames = gsub(" ", "_", columnNames)

      # create a dataframe with first colum and append the rest (workaround)
      df = data.frame(columnValues[[1]])

      for (i in 2:length(columnValues)) {
          df[i] = columnValues[[i]]
      }

      names(df) = columnNames
      return(df)
      
  } else {
      return(data.frame())
  }
}

# Testing Section

In [22]:
#scrapeGame(year = '2009', country = 'England', club_id = 3299, game_name = "weltmeisterschaft [0-9]+")

scrapeGameByYear = function(year, country) {
    
    df_wm %>%
        filter(Club == country) -> tempDf
    
    tempDf$club_id -> club_id
    
    #pmap_df(list('2015', 'England', '3299', c('weltmeisterschaft [0-9]+', 'freundschaftsspiele', 'em-qualifikation', 'wm-qualifikation europa', 'europameisterschaft [0-9]+')), scrapeGame)
    return(pmap_df(list(year, country, club_id,
                        c(
                            #'world cup [0-9]+',
                            #'international friendlies'
                            #'european qualifiers',
                            'world cup qualification africa'
                            #'euro [0-9]+'
                            #'copa'
                            #'confederations cup [0-9]+'
                            #'ostasienmeisterschaft',
                            #'wm-qualifikation asien',
                            #'asienmeisterschaft [0-9]+',
                            #'asienmeisterschaft qualifikation'
                            #'africa cup of nations'
                            #'fifa arab cup',
                            #'gold cup [0-9]+'
                        )), scrapeGame))
}

In [19]:
scrapeSomething = function(country) {
    return(map2_df(seq('2009', '2021', by=1), country, scrapeGameByYear))
}

## Scrape world cup & friendlies for all 32 countries

In [23]:
complete_df = map_df(teilnehmer, scrapeSomething)

[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2009"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2010"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2011"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2012"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2013"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2014"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2015"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2016"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2017"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2018"
[1] 200
[1] "https://www.transfermarkt.com/Engla

In [24]:
complete_df

Matchday,Date,Time,Venue,Ranking,Opponent,System_of_play,Attendance,Result,game_name,country
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
group J,"Sat Jun 2, 2012",8:00 PM,H,,Liberia,4-4-2 double 6,15.000,3:1,World Cup qualification Africa,Senegal
group J,"Sat Jun 9, 2012",3:00 PM,A,,Uganda,4-4-2 double 6,30.000,1:1,World Cup qualification Africa,Senegal
group J,"Sat Mar 23, 2013",6:00 PM,H,,Angola,4-3-3 Attacking,13.500,1:1,World Cup qualification Africa,Senegal
group J,"Sat Jun 8, 2013",5:00 PM,A,,Angola,4-3-3 Attacking,40.000,1:1,World Cup qualification Africa,Senegal
group J,"Sun Jun 16, 2013",6:00 PM,A,,Liberia,4-4-2 double 6,35.000,0:2,World Cup qualification Africa,Senegal
group J,"Sat Sep 7, 2013",10:00 PM,H,,Uganda,4-4-2 double 6,2.000,1:0,World Cup qualification Africa,Senegal
3rd round 1st leg,"Sat Oct 12, 2013",7:00 PM,A,,Ivory Coast,4-5-1 flat,30.000,3:1,World Cup qualification Africa,Senegal
3rd round 2nd leg,"Sat Nov 16, 2013",8:00 PM,H,,Ivory Coast,4-4-2 double 6,15.000,1:1,World Cup qualification Africa,Senegal
Second Round 1st leg,"Fri Nov 13, 2015",12:30 PM,A,,Madagascar,4-4-2 double 6,24.000,2:2,World Cup qualification Africa,Senegal


# Split Result in Goals

In [16]:
#df_football %>%
#    mutate(goals_scored_england = ternary_v(strsplit_custom_v(Ergebnis, ':', 1), strsplit_custom_v(Ergebnis, ':', 2), (Heimmannschaft=='England'))) -> df_football_goals
#df_football_goals

## We save all data for the WorldCups&friendlies as csv

In [25]:
#complete_df
write.csv2(complete_df, file='./world_cup_africa_qualifications_football_en.csv', row.names = F)

## Modelling - Data Cleaning and Transformation

In [143]:
data_set = read.csv('https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/world_cup%26friendlies_football_en.csv', sep=';')
fifa_ranking = read.csv("https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/FIFA_Ranking.csv", header = TRUE, sep = ",", stringsAsFactors=FALSE, fileEncoding="latin1")


In [144]:
fifa_ranking %>%
    separate(Rang, c('Rang', 'Sym')) %>%
    select(-'Sym') %>%
    mutate(fifa_date = ymd(Datum)) %>%
    select(-c('Datum')) -> fifa_ranking1
fifa_ranking1

date_list = unique(fifa_ranking1$fifa_date)
date_list

Land,Rang,Punkte,fifa_date
<chr>,<chr>,<chr>,<date>
Spain,1,1588,2009-09-02
Netherlands,2,1376,2009-09-02
Germany,3,1186,2009-09-02
Italy,4,1186,2009-09-02
Russia,5,1129,2009-09-02
England,6,1127,2009-09-02
Argentina,7,1113,2009-09-02
Croatia,8,1101,2009-09-02
France,9,1040,2009-09-02


[1] "2009-09-02" "2009-10-16" "2009-11-20" "2009-12-16" "2010-02-03"
  [6] "2010-03-03" "2010-03-31" "2010-04-28" "2010-05-26" "2010-07-14"
 [11] "2010-08-11" "2010-09-24" "2010-10-20" "2010-11-17" "2010-12-15"
 [16] "2011-01-12" "2011-02-02" "2011-03-09" "2011-04-13" "2011-05-18"
 [21] "2011-06-29" "2011-07-27" "2011-08-24" "2011-09-21" "2011-10-19"
 [26] "2011-11-23" "2011-12-21" "2012-01-18" "2012-02-15" "2012-03-07"
 [31] "2012-04-11" "2012-05-09" "2012-06-06" "2012-07-04" "2012-08-08"
 [36] "2012-09-05" "2012-10-03" "2012-11-07" "2012-12-19" "2013-01-17"
 [41] "2013-02-14" "2013-03-14" "2013-04-11" "2013-05-09" "2013-06-06"
 [46] "2013-07-04" "2013-08-08" "2013-09-12" "2013-10-17" "2013-11-28"
 [51] "2013-12-19" "2014-01-16" "2014-02-13" "2014-03-13" "2014-04-10"
 [56] "2014-05-08" "2014-07-17" "2014-08-14" "2014-09-18" "2014-10-23"
 [61] "2014-11-27" "2014-12-18" "2015-02-12" "2015-03-12" "2015-04-09"
 [66] "2015-05-07" "2015-06-04" "2015-07-09" "2015-08-06" "2015-09-03"
 [71] "2015-10-01" "2015-11-05" "2015-12-03" "2016-01-07" "2016-02-04"
 [76] "2016-03-03" "2016-04-07" "2016-05-05" "2016-05-20" "2016-06-02"
 [81] "2016-07-14" "2016-08-11" "2016-09-15" "2016-10-20" "2016-11-24"
 [86] "2016-12-22" "2017-01-12" "2017-02-09" "2017-03-09" "2017-04-06"
 [91] "2017-05-04" "2017-06-01" "2017-07-06" "2017-08-10" "2017-09-14"
 [96] "2017-10-16" "2017-11-23" "2017-12-21" "2018-01-18" "2018-02-15"
[101] "2018-03-15" "2018-04-12" "2018-05-17" "2018-06-07" "2018-08-16"
[106] "2018-09-20" "2018-10-25" "2018-11-29" "2018-12-20" "2019-02-08"
[111] "2019-04-04" "2019-07-25" "2019-09-10" "2019-09-11" "2019-10-24"
[116] "2019-11-28" "2020-02-20" "2020-03-24" "2020-07-23" "2020-09-17"
[121] "2020-10-22" "2020-11-27" "2020-12-10" "2021-02-03" "2021-02-18"
[126] "2021-06-15" "2021-08-12" "2021-10-20" "2021-10-21" "2021-11-19"
[131] "2021-12-23" "2022-02-10" "2022-03-31" "2022-06-06" "2022-06-23"

In [145]:
data_set

Matchday,Date,Time,Venue,Ranking,Opponent,System_of_play,Attendance,Result,game_name,country
<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Group C,"Sat Jun 12, 2010",8:30 PM,H,NA,United States,4-4-2 double 6,38.646,1:1,World Cup 2010,England
Group C,"Fri Jun 18, 2010",8:30 PM,H,NA,Algeria,4-4-2 double 6,64.100,0:0,World Cup 2010,England
Group C,"Wed Jun 23, 2010",4:00 PM,A,NA,Slovenia,4-4-2 double 6,36.893,0:1,World Cup 2010,England
Round of 16,"Sun Jun 27, 2010",4:00 PM,A,NA,Germany,4-4-2 double 6,40.510,4:1,World Cup 2010,England
NA,"Wed Mar 3, 2010",9:00 PM,H,NA,Egypt,4-4-2 double 6,80.602,3:1,International Friendlies,England
NA,"Mon May 24, 2010",9:00 PM,H,NA,Mexico,4-2-3-1,88.638,3:1,International Friendlies,England
NA,"Sun May 30, 2010",2:15 PM,H,NA,Japan,4-4-2 double 6,15.326,2:1,International Friendlies,England
NA,"Wed Aug 11, 2010",9:00 PM,H,NA,Hungary,4-2-3-1,72.024,2:1,International Friendlies,England
NA,"Wed Nov 17, 2010",9:00 PM,H,NA,France,4-2-3-1,85.495,1:2,International Friendlies,England


In [146]:
data_set_reduced = data_set %>%
    select(c('Date', 'Venue', 'Opponent', 'Result', 'game_name', 'country'))

In [6]:
lubridate::as_date('Jun 12, 2010')
lubridate:as_datetime('Jun 12, 2010')

Warning message:
“All formats failed to parse. No formats found.”


[1] NA

ERROR: Error in eval(expr, envir, enclos): object 'lubridate' not found


## Obtain goals of enemy and country

In [147]:
data_set_reduced %>%
    separate(Result, c('result_left', 'result_right')) %>%
    mutate(goals_country = ternary_v(result_left, result_right, Venue=='H')) %>%
    mutate(goals_enemy = ternary_v(result_left, result_right, Venue=='A')) -> data_set_increased

data_set_increased

Warning message:
“Expected 2 pieces. Additional pieces discarded in 38 rows [51, 53, 212, 304, 307, 420, 474, 531, 533, 534, 589, 617, 618, 660, 693, 832, 833, 839, 840, 841, ...].”


Date,Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Sat Jun 12, 2010",H,United States,1,1,World Cup 2010,England,1,1
"Fri Jun 18, 2010",H,Algeria,0,0,World Cup 2010,England,0,0
"Wed Jun 23, 2010",A,Slovenia,0,1,World Cup 2010,England,1,0
"Sun Jun 27, 2010",A,Germany,4,1,World Cup 2010,England,1,4
"Wed Mar 3, 2010",H,Egypt,3,1,International Friendlies,England,3,1
"Mon May 24, 2010",H,Mexico,3,1,International Friendlies,England,3,1
"Sun May 30, 2010",H,Japan,2,1,International Friendlies,England,2,1
"Wed Aug 11, 2010",H,Hungary,2,1,International Friendlies,England,2,1
"Wed Nov 17, 2010",H,France,1,2,International Friendlies,England,1,2


In [148]:
convert_month_to_decimal = function(month) {
    num = 0
    
    for (i in 1:12) {
        if(month == switch(i,
                          'Jan',
                          'Feb',
                          'Mar',
                          'Apr',
                          'May',
                          'Jun',
                          'Jul',
                          'Aug',
                          'Sep',
                          'Oct',
                          'Nov',
                          'Dec')
          ) {
            num = i
            break;
        }
    }
    return(i)
}

convert_month_to_decimal_v = Vectorize(convert_month_to_decimal)

In [149]:
data_set_increased %>%
    separate(Date, c('weekday', 'month', 'day', 'year'), sep=' ', remove=F) %>%
    separate(day, c('day')) %>%
    mutate(month = convert_month_to_decimal_v(month)) %>%
    mutate(date_of_play = ymd(paste0(year, '-', month, '-', day))) %>%
    select(-c('Date', 'weekday', 'month', 'day', 'year')) -> data_set_cleaned1

data_set_cleaned1

Warning message:
“Expected 1 pieces. Additional pieces discarded in 2399 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy,date_of_play
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>
H,United States,1,1,World Cup 2010,England,1,1,2010-06-12
H,Algeria,0,0,World Cup 2010,England,0,0,2010-06-18
A,Slovenia,0,1,World Cup 2010,England,1,0,2010-06-23
A,Germany,4,1,World Cup 2010,England,1,4,2010-06-27
H,Egypt,3,1,International Friendlies,England,3,1,2010-03-03
H,Mexico,3,1,International Friendlies,England,3,1,2010-05-24
H,Japan,2,1,International Friendlies,England,2,1,2010-05-30
H,Hungary,2,1,International Friendlies,England,2,1,2010-08-11
H,France,1,2,International Friendlies,England,1,2,2010-11-17


In [150]:
test_date = function(date_param) {
    for(i in 1:length(date_list)) {
        if(date_list[i] > date_param) {
            break;
        }
    }
    if(i != 1) {
        return(date_list[i-1])
    } else {
        return(date_list[1])
    }
}

test_date_v = Vectorize(test_date)

In [151]:
data_set_cleaned1 %>%
    mutate(fifa_date = as_date(test_date_v(date_of_play))) -> data_set_fifa_date

In [152]:
data_set_fifa_date

Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy,date_of_play,fifa_date
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>
H,United States,1,1,World Cup 2010,England,1,1,2010-06-12,2010-05-26
H,Algeria,0,0,World Cup 2010,England,0,0,2010-06-18,2010-05-26
A,Slovenia,0,1,World Cup 2010,England,1,0,2010-06-23,2010-05-26
A,Germany,4,1,World Cup 2010,England,1,4,2010-06-27,2010-05-26
H,Egypt,3,1,International Friendlies,England,3,1,2010-03-03,2010-03-03
H,Mexico,3,1,International Friendlies,England,3,1,2010-05-24,2010-04-28
H,Japan,2,1,International Friendlies,England,2,1,2010-05-30,2010-05-26
H,Hungary,2,1,International Friendlies,England,2,1,2010-08-11,2010-08-11
H,France,1,2,International Friendlies,England,1,2,2010-11-17,2010-11-17


## join both data frames using fifa_date

In [153]:
merge(data_set_fifa_date, fifa_ranking1, by.x=c('country', 'fifa_date'), by.y=c('Land', 'fifa_date')) -> newest_df

In [154]:
getFifaRanking = function(country, date) {
    fifa_ranking1 %>%
        filter(Land == country) %>%
        filter(fifa_date == date) %>%
        pluck(2) -> temp
    #print(as.numeric(unlist(temp)))
    
    as.numeric(temp)
}

getFifaRanking_v = Vectorize(getFifaRanking)

In [18]:
#fifa_ranking1 %>%
#    filter(Land == 'Costa Rica') %>%
#    filter(fifa_date == '2009-12-16')
getFifaRanking('Germany', '2022-06-06')

[1] 12

In [155]:
newest_df %>% 
    mutate(rank_opponent = getFifaRanking_v(Opponent, fifa_date)) -> somethingDf


In [156]:
somethingDf

country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,Rang,Punkte,rank_opponent
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>,<chr>,<named list>
Argentina,2009-12-16,H,Costa Rica,3,2,International Friendlies,3,2,2010-01-27,8,1085,44
Argentina,2010-02-03,H,Jamaica,2,1,International Friendlies,2,1,2010-02-11,8,1082,78
Argentina,2010-03-03,A,Germany,0,1,International Friendlies,1,0,2010-03-03,9,1087,5
Argentina,2010-04-28,H,Haiti,4,0,International Friendlies,4,0,2010-05-05,7,1084,87
Argentina,2010-04-28,H,Canada,5,0,International Friendlies,5,0,2010-05-24,7,1084,63
Argentina,2010-05-26,H,Nigeria,1,0,World Cup 2010,1,0,2010-06-12,7,1076,21
Argentina,2010-05-26,H,South Korea,4,1,World Cup 2010,4,1,2010-06-17,7,1076,47
Argentina,2010-05-26,A,Greece,0,2,World Cup 2010,2,0,2010-06-22,7,1076,13
Argentina,2010-05-26,H,Mexico,3,1,World Cup 2010,3,1,2010-06-27,7,1076,17


In [157]:
toNumber = function(value) {
    return(value[1])
}

toNumber_v = Vectorize(toNumber)

## Functions for Venue transformation

In [158]:
isNeutral = function(country, opponent, game_name) {
    
    HOSTCOUNTRY = NULL
    
    if(game_name == 'World Cup 2010') {
        HOSTCOUNTRY = 'South Africa'
    } else if(game_name == 'World Cup 2014') {
        HOSTCOUNTRY = 'Brazil'
    } else if(game_name == 'World Cup 2018') {
        HOSTCOUNTRY = 'Russia'
    } else if(game_name == 'World Cup 2022') {
        HOSTCOUNTRY = 'Qatar'
    } 
    
    if (!is.null(HOSTCOUNTRY)) {
        venue = 'N'
        if(country != HOSTCOUNTRY & opponent != HOSTCOUNTRY) {
            return('N')
        } else {
            return('H/A')
        }
    } else {
        return ('H/A')
    }
}
    
isNeutral_v = Vectorize(isNeutral)

In [159]:
isNeutral('Germany', 'Brazil', 'International Friendlies')

[1] "H/A"

In [160]:
somethingDf %>%
    mutate(rank_opponent = toNumber_v(rank_opponent)) -> test

unique(test$game_name)
test

[1] "International Friendlies" "World Cup 2010"          
[3] "World Cup 2014"           "World Cup 2018"          
[5] "World Cup 2022"

country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,Rang,Punkte,rank_opponent
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>
Argentina,2009-12-16,H,Costa Rica,3,2,International Friendlies,3,2,2010-01-27,8,1085,44
Argentina,2010-02-03,H,Jamaica,2,1,International Friendlies,2,1,2010-02-11,8,1082,78
Argentina,2010-03-03,A,Germany,0,1,International Friendlies,1,0,2010-03-03,9,1087,5
Argentina,2010-04-28,H,Haiti,4,0,International Friendlies,4,0,2010-05-05,7,1084,87
Argentina,2010-04-28,H,Canada,5,0,International Friendlies,5,0,2010-05-24,7,1084,63
Argentina,2010-05-26,H,Nigeria,1,0,World Cup 2010,1,0,2010-06-12,7,1076,21
Argentina,2010-05-26,H,South Korea,4,1,World Cup 2010,4,1,2010-06-17,7,1076,47
Argentina,2010-05-26,A,Greece,0,2,World Cup 2010,2,0,2010-06-22,7,1076,13
Argentina,2010-05-26,H,Mexico,3,1,World Cup 2010,3,1,2010-06-27,7,1076,17


In [161]:
test %>%
    mutate(isNeutral = isNeutral_v(country, Opponent, game_name)) %>%
    mutate(Venue = ifelse(isNeutral == 'N', 'N', Venue)) %>%
    select(-c('isNeutral', 'Punkte')) -> final_data_set

In [162]:
final_data_set %>%
    filter(result_left == "") %>%
    filter(game_name == 'World Cup 2022') -> world_cup_2022

final_data_set %>%
    filter(result_left != "") -> data_set_without_wc2022

data_set_without_wc2022 %>%
    mutate(goals_country = as.numeric(goals_country)) %>%
    mutate(Rank = as.numeric(Rang)) %>%
    mutate(goals_country = as.numeric(goals_country)) %>%
    select('country', 'goals_country', 'rank_opponent', 'Rank', 'Venue') -> data_set_without_wc2022_cleaned

In [163]:
data_set_without_wc2022_cleaned

# turn Venue into dummy variables
data_set_without_wc2022_cleaned = dummy_cols(data_set_without_wc2022_cleaned, select_columns = 'Venue')

data_set_without_wc2022_cleaned

data_set_without_wc2022_cleaned %>%
    filter(country == 'Brazil') -> histogramm

country = histogramm$goals_country

mean(country)

var(country)

country,goals_country,rank_opponent,Rank,Venue
<chr>,<dbl>,<dbl>,<dbl>,<chr>
Argentina,3,44,8,H
Argentina,2,78,8,H
Argentina,1,5,9,A
Argentina,4,87,7,H
Argentina,5,63,7,H
Argentina,1,21,7,N
Argentina,4,47,7,N
Argentina,2,13,7,N
Argentina,3,17,7,N


country,goals_country,rank_opponent,Rank,Venue,Venue_A,Venue_H,Venue_N
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>
Argentina,3,44,8,H,0,1,0
Argentina,2,78,8,H,0,1,0
Argentina,1,5,9,A,1,0,0
Argentina,4,87,7,H,0,1,0
Argentina,5,63,7,H,0,1,0
Argentina,1,21,7,N,0,0,1
Argentina,4,47,7,N,0,0,1
Argentina,2,13,7,N,0,0,1
Argentina,3,17,7,N,0,0,1


[1] 2.160377

[1] 2.574034

## Split data into test and training data set

In [164]:
sample = initial_split(data_set_without_wc2022_cleaned, prop=0.7)
train = training(sample)
test = testing(sample)

In [165]:
test

,country,goals_country,rank_opponent,Rank,Venue,Venue_A,Venue_H,Venue_N
,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>
4,Argentina,4,87,7,H,0,1,0
11,Argentina,1,37,5,A,1,0,0
12,Argentina,4,1,5,H,0,1,0
16,Argentina,4,63,4,H,0,1,0
22,Argentina,1,71,5,A,1,0,0
24,Argentina,1,44,9,A,1,0,0
25,Argentina,3,38,9,H,0,1,0
27,Argentina,0,7,10,A,1,0,0
28,Argentina,3,16,11,A,1,0,0


In [166]:
train

model = 
    glm(
        goals_country ~ rank_opponent + Venue_H + Venue_N + Rank,
        train,
        family = poisson(link = 'log')
       )
summary(model)

,country,goals_country,rank_opponent,Rank,Venue,Venue_A,Venue_H,Venue_N
,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>
156,Belgium,4,29,15,A,1,0,0
1910,Switzerland,0,38,24,N,0,0,1
775,France,3,30,16,N,0,0,1
2004,Tunisia,0,49,37,A,1,0,0
2111,United States,1,31,24,A,1,0,0
1017,Iran,2,93,30,A,1,0,0
666,Ecuador,0,63,58,A,1,0,0
533,Croatia,0,119,16,A,1,0,0
2153,Uruguay,1,4,7,A,1,0,0



Call:
glm(formula = goals_country ~ rank_opponent + Venue_H + Venue_N + 
    Rank, family = poisson(link = "log"), data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.8995  -1.2110  -0.1704   0.5560   3.7955  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    0.1584666  0.0483629   3.277  0.00105 ** 
rank_opponent  0.0076364  0.0005173  14.761  < 2e-16 ***
Venue_H        0.2872618  0.0454267   6.324 2.55e-10 ***
Venue_N        0.1906142  0.0739947   2.576  0.00999 ** 
Rank          -0.0093364  0.0008958 -10.422  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 2237.8  on 1549  degrees of freedom
Residual deviance: 1921.2  on 1545  degrees of freedom
  (22 observations deleted due to missingness)
AIC: 4827.3

Number of Fisher Scoring iterations: 5


In [185]:
# Qatar gegen Niederlande

wales = data.frame(rank_opponent = 15, Venue_H = 0, Venue_N= 1, Rank = 18)
usa = data.frame(rank_opponent = 18, Venue_H = 0, Venue_N= 1, Rank = 15)

tore_wales = predict(model, newdata = wales, type = "response")
tore_usa = predict(model, newdata = usa, type = "response")

tore_wales
tore_usa

# Poisson Regression anhand des formels f(x) = (lambda^x)*e^(-lambda)/x!
# x = 0 für den USA und x = 0 für Wales
P_Wales = exp(-tore_wales)
P_USA = exp(-tore_usa)
P = P_Wales*P_USA
P_Wales
P_USA
P

for (i in 0:8) {
    print(ppois(0, tore_wales, lower=F) * ppois(i, tore_usa, lower=F))
}

1 
1.343895

1 
1.414096

1 
0.2608278

1 
0.2431453

1 
0.06341906

[1] 0.5594459
[1] 0.3052958
[1] 0.1255994
[1] 0.04089674
[1] 0.01095232
[1] 0.00248346
[1] 0.0004874971
[1] 8.428507e-05
[1] 1.301251e-05
